### Total Factor Productivity from BLS

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
url = ('https://download.bls.gov/pub/time.series/mp/'+
       'mp.data.1.AllData')
df = pd.read_table(url, sep='\s+', skipinitialspace=True)

In [3]:
d = {'TFP': '01', 'Cap': '15', 'Prod': '06', 'Lab': '18'}
d = {f'MPU4910{v}3': k for k, v in d.items()}
res = (df.loc[df['series_id'].isin(d.keys())]
         .set_index(['year', 'series_id'])
         .value.unstack().rename(d, axis=1))

res.index = [pd.to_datetime(f'{y}-07-01') for y in res.index]
res.loc['1989':].to_csv(data_dir / 'tfp.csv', index_label='date')

ltdt = dtxt(res.index[-1])['year']
lt = res.iloc[-1]
tot = value_text(lt.Prod, 'increase_by')
o = ('(see \\begin{tikzpicture} \\node[line width=0.3mm, circle, '+
     'draw=black, scale=0.4, aspect=0.8] (d) at (0,0) {}; '+
     '\end{tikzpicture})')
cap = value_text(lt.Cap, 'contribution', 'pp', threshold=0.1)
lab = value_text(lt.Lab, 'contribution', 'pp', threshold=0.1)
tfp = value_text(lt.TFP, 'contribution', 'pp', threshold=0.1, casual=True)
colors = {'Cap': 'violet!60!white', 'Lab': 'blue!60!black', 
          'TFP': 'cyan!40!white'}
cb = {name: c_box(col) for name, col in colors.items()}

text = (f'In {ltdt}, labor productivity {tot} {o}. Capital '+
        f'intensity {cap} {cb["Cap"]}, and labor composition '+
        f'{lab} {cb["Lab"]}. The remainder, called total factor '+
        f'productivity, {tfp} {cb["TFP"]}.')
write_txt(text_dir / 'tfp.txt', text)
print(text)

In 2021, labor productivity increased by 2.4 percent (see \begin{tikzpicture} \node[line width=0.3mm, circle, draw=black, scale=0.4, aspect=0.8] (d) at (0,0) {}; \end{tikzpicture}). Capital intensity subtracted 1.1 percentage points (see \cbox{violet!60!white}), and labor composition subtracted 0.5 percentage point (see \cbox{blue!60!black}). The remainder, called total factor productivity, added 4.1 percentage points (see \cbox{cyan!40!white}).
